In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'w'
import warnings
warnings.filterwarnings('ignore')

In [4]:
cols = ['JD', 'Magnitude']
df = pd.read_csv('data/aavsodata_betelgeuse.txt', usecols=cols)

In [5]:
df.head()

,JD,Magnitude
0,2419789.5,1.1
1,2421597.3,1.0
2,2421602.4,0.8
3,2421607.3,0.9
4,2421612.3,0.8


In [28]:
from astropy.time import Time
from datetime import datetime

In [35]:
def jd_date(jd):
    t_iso = Time(jd, format='jd', scale='utc').iso
    dt = datetime.strptime(t_iso, '%Y-%m-%d %H:%M:%S.%f')
    return dt.date()

In [36]:
for idx in range(31134-5, 31134):
    jd = df['JD'][idx]
    print(idx, jd, jd_date(jd))

31129 2458841.26944 2019-12-23
31130 2458841.40278 2019-12-23
31131 2458841.404 2019-12-23
31132 2458841.7483 2019-12-24
31133 2458841.7487 2019-12-24


In [39]:
df['date'] = df['JD'].apply(jd_date)

In [40]:
df.head()

,JD,Magnitude,date
0,2419789.5,1.1,1913-01-22
1,2421597.3,1.0,1918-01-03
2,2421602.4,0.8,1918-01-08
3,2421607.3,0.9,1918-01-13
4,2421612.3,0.8,1918-01-18


In [43]:
len(df[df.duplicated(['date'])])

16791